<br>
<br>

# **Modelos del lenguaje basados en redes neuronales artificiales**

## **Redes neuronales recurrentes (RNN)**

### **Ejemplo de aplicación de un modelo LSTM a la clasificación de textos**

#### **Dataset**

El conjunto de datos "AG_NEWS" es un conjunto de datos de clasificación de texto ampliamente utilizado en el campo del procesamiento de lenguaje natural (NLP). Contiene noticias de diferentes categorías y se utiliza comúnmente para tareas de clasificación de texto. El conjunto de datos AG_NEWS consta de noticias de cuatro categorías principales, que son:

1. **World**: Noticias sobre eventos y acontecimientos globales, como política internacional, relaciones internacionales y noticias mundiales en general.

2. **Sports**: Noticias relacionadas con eventos deportivos, resultados de partidos, eventos deportivos nacionales e internacionales, etc.

3. **Business**: Noticias relacionadas con el mundo de los negocios, finanzas, economía, empresas, informes de ganancias y otros temas económicos.

4. **Sci/Tech**: Noticias relacionadas con ciencia y tecnología, incluyendo avances científicos, novedades tecnológicas, gadgets, investigaciones científicas y más.

Cada instancia del conjunto de datos AG_NEWS generalmente consiste en un título y un cuerpo de una noticia, junto con una etiqueta que indica la categoría a la que pertenece. 

In [3]:
from torchtext import datasets
from torchtext.data import to_map_style_dataset
import numpy as np

# Load the dataset
train_iter, test_iter = datasets.AG_NEWS(split=('train', 'test'))

train_ds = to_map_style_dataset(train_iter)
test_ds = to_map_style_dataset(test_iter)

train = np.array(train_ds)
test = np.array(test_ds)

#### **Tokenización**

In [4]:
# Create vocabulary and embedding

from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer("basic_english")

vocab = build_vocab_from_iterator(map(lambda x: tokenizer(x[1]), train_iter), specials=['<pad>','<unk>'])
vocab.set_default_index(vocab["<unk>"])

In [5]:
print("Tamaño del vocabulario:", len(vocab), "tokens")
print("Tokenización de la frase 'Here is an example sentence':", tokenizer("Here is an example sentence"))
print("Índices de las palabras 'here', 'is', 'an', 'example', 'supercalifragilisticexpialidocious':", vocab(['here', 'is', 'an', 'example', 'supercalifragilisticexpialidocious']))
print("Palabras correspondientes a los índices 475, 21, 30, 5297, 0:", vocab.lookup_tokens([475, 21, 30, 5297, 0]))
print("Las diez primeras palabras del vocabulario:", vocab.get_itos()[:10])

Tamaño del vocabulario: 95812 tokens
Tokenización de la frase 'Here is an example sentence': ['here', 'is', 'an', 'example', 'sentence']
Índices de las palabras 'here', 'is', 'an', 'example', 'supercalifragilisticexpialidocious': [476, 22, 31, 5298, 1]
Palabras correspondientes a los índices 475, 21, 30, 5297, 0: ['version', 'at', 'from', 'establish', '<pad>']
Las diez primeras palabras del vocabulario: ['<pad>', '<unk>', '.', 'the', ',', 'to', 'a', 'of', 'in', 'and']


In [6]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

print("Tokenización de la frase 'Here is an example sentence':", text_pipeline("Here is an example sentence"))

Tokenización de la frase 'Here is an example sentence': [476, 22, 31, 5298, 2994]


#### **DataLoader**

En PyTorch, **DataLoader** se utiliza para cargar y manejar datos de manera eficiente durante el entrenamiento de modelos de aprendizaje profundo, especialmente en tareas de aprendizaje supervisado como la clasificación, la regresión y más. El DataLoader forma parte de la biblioteca torch.utils.data, y su objetivo principal es facilitar la administración de lotes (batches) de datos y la distribución de esos lotes al modelo de forma automática.

La función <code>collate_batch</code> es una función personalizada que se utiliza como argumento para collate_fn al crear instancias de los objetos DataLoader. Tiene la responsabilidad de procesar y agrupar las muestras individuales de datos dentro de un lote (batch) de manera que sean compatibles para su posterior procesamiento dentro de la LSTM. Es especialmente útil cuando las secuencias de texto tienen longitudes diferentes y es necesario realizar un relleno (padding) para que todas tengan la misma longitud.

In [7]:
from torch.utils.data import DataLoader
import torch


def collate_batch(batch):
    label_list, text_list = [], []
    for sample in batch:
        label, text = sample
        text_list.append(torch.tensor(text_pipeline(text), dtype=torch.long))
        label_list.append(label_pipeline(label))
    return torch.tensor(label_list, dtype=torch.long), torch.nn.utils.rnn.pad_sequence(text_list, batch_first=True, padding_value=vocab["<pad>"])

train_dataloader = DataLoader(
    train_iter, batch_size=64, shuffle=True, collate_fn=collate_batch
)

test_dataloader = DataLoader(
    test_iter, batch_size=64, shuffle=True, collate_fn=collate_batch
)

Para verificar que estamos creando correctamente los lotes, vamos a imprimir las primeras cuatro instancias:

In [8]:
for batch in train_dataloader:
    print(batch[1][:4])
    print("\n")
    print(batch[0][:4])
    print("\n")
    break

tensor([[ 1088,     5,  3354,    26,    12,  6883,   548,   247,  6883,   548,
           247, 36936,    88,     2,    29,   736,     6,  3286,   484,    19,
          1866,     5,  3354,    23,    26,  3007,     9,   413,    69,   205,
             4,   163,   333,   596,   359,     2,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [ 2422,  8668,    11,   120,     7, 18932,    14,    32,    15,    32,
            16,     5,    38,    79,     2,   109,     8,     3,    51,    22,
            62,  1906,     4,    46,     5,   110,     6,   198,     4,    18,
            17,    10,   184,   508,   131, 15173,    12,     2,    18,    17,
            10,   184

#### **Modelo LSTM**

Definimos una clase llamada `LSTMTextClassificationModel` para la clasificación de texto basado en LSTM. Esta clase hereda de `nn.Module`, la clase base para todos los modelos en PyTorch.

**Inicialización del Modelo**: En el método `__init__`, se definen los componentes principales del modelo y se configuran sus parámetros:
   - **vocab_size**: El tamaño del vocabulario, es decir, la cantidad de palabras únicas en el conjunto de datos de entrenamiento.
   - **embed_dim**: La dimensión de los vectores de embedding para representar las palabras.
   - **hidden_dim**: El tamaño de la capa oculta de la LSTM, que controla la cantidad de unidades o "memoria" en la red LSTM.
   - **num_class**: El número de clases de salida en la tarea de clasificación.


**Capa de Embedding**: Se define una capa de embedding (`nn.Embedding`) que se utilizará para representar las palabras como vectores densos. Esta capa no es pre-entrenada, lo que significa que los vectores de embedding se entrenarán junto con el modelo durante el proceso de entrenamiento.

**Capa LSTM**: Se define una capa LSTM (`nn.LSTM`) que toma los vectores de embedding como entrada. `embed_dim` es la dimensión de entrada de la capa LSTM, y `hidden_dim` es la dimensión de su capa oculta. `batch_first=True` indica que los datos de entrada tendrán la forma `(batch_size, sequence_length, embed_dim)`.

**Capa Fully Connected (FC)**: Se define una capa completamente conectada (`nn.Linear`) que se utiliza para producir las salidas de clasificación. Toma las salidas de la capa LSTM correspondientes a la última iteración y las reduce a `num_class` dimensiones, que corresponde al número de clases en la tarea de clasificación.


In [9]:
import torch
import torch.nn as nn

class LSTMTextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super(LSTMTextClassificationModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)  # <-- Capa de embedding genérica (no pre-entrenada)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_class)

    def forward(self, text):
        embedded = self.embedding(text)  # <-- Tras pasar por la capa de embedding, las palabras se representan como vectores
        lstm_out, _ = self.lstm(embedded)
        # Tomar la última salida de la secuencia LSTM
        last_output = lstm_out[:, -1, :]
        output = self.fc(last_output)
        return output


Antes de seguir veamos cómo se conforma un *batch* de datos tras pasar la capa de *embedding*. Como se muestra en la figura siguiente, un *batch* es un vector de tres dimensiones, donde la primera dimensión es el tamaño del *batch*, la segunda dimensión es el número de palabras en cada secuencia y la tercera dimensión es el tamaño del vector de *embedding* (*channels* o *features*).

<img src="imgs/Lote1_b.svg" width="30%">

Por ejemplo, un *token* estaría ahora representado por un vector de *embedding* de tres componentes, en lugar de un escalar. Si lo quisiéramos referenciar sería: <code>mini_lote[0,3,:]</code>.

<img src="imgs/Lote2_b.svg" width="30%">


Fíjate que la tercera dimensión del tensor de salida de la LSTM no tiene por qué tener el mismo tamaño que la tercera dimensión del tensor de entrada. La parte del tensor correspondiente a la línea de código <code>last_output = lstm_out[:, -1, :]</code> sería parecida a la siguiente:

<img src="imgs/Lote3_b.svg" width="30%">

Ya podemos pasar un primer mini-lote de datos por la red. Para ello, vamos a crearlo a partir de los datos de entrenamiento. Esto solo lo hacemos para asegurarnos de que el código funciona correctamente.

In [10]:
model = LSTMTextClassificationModel(len(vocab), 32, 64, 4)
model.train()

for batch in train_dataloader:
    predicted_label = model(batch[1])
    label = batch[0]
    break

print(batch[1][:4])
print(predicted_label[:4])
print(label[:4])

/home/adrian/miniconda3/envs/PLN/lib/python3.11/site-packages/torch/utils/data/datapipes/iter/combining.py:333: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


tensor([[ 4448,  1303,   105,   446,  1384,   648,    14,    28,    15,    28,
            16,  4448,  1303,    14, 18883,     2,   138,    15,     4,     3,
            79,     2,   121,    52,     2,    10,     2, 35100,  1080,    55,
             4,    11,    57,    27,    26,   105,   446, 43845,    30,     3,
            52,     2,    10,     2,  1018,     9,   688,   486, 70003,  3816,
             7,  1018,  2373,     2,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0],
        [    2,     2,     2,   508,    13,   902,   638,   223,   717,     3,
          6651,    64,  6029,  1104,  6751,    22,  3391,    26,    96,   794,
           635,   390,     2,     9

#### **Entrenamiento**

Vamos a construir las funciones <code>train</code> y <code>evaluate</code>. Dentro de la función <code>evaluate</code> vamos a prestar atención al contexto <code>with torch.no_grad():</code>, que sirve para indicar que no se van a calcular los gradientes. Esto es así porque en la fase de evaluación no se van a actualizar los pesos de la red, solo se van a utilizar para calcular la precisión de la red. Esto optimiza los recursos utilizados por PyTorch en cuanto a la gestión de la memoria.

Recuerda que en PyTorch, `model.train()` y `model.eval()` son métodos que se utilizan para cambiar el modo de entrenamiento de un modelo de aprendizaje profundo. Estos métodos afectan el comportamiento de ciertos módulos en el modelo, como las capas de dropout y normalización, que se comportan de manera diferente durante el entrenamiento y la evaluación. Específicamente hacen lo siguiente:

**model.train()**:
   - Cuando se llama a `model.train()`, el modelo se coloca en modo de entrenamiento.
   - En este modo, las capas de dropout se activan, lo que significa que se aplican durante el entrenamiento para ayudar a evitar el sobreajuste. Durante el entrenamiento, las capas de dropout "apagan" aleatoriamente un porcentaje de las unidades (neuronas) en la red en cada paso de entrenamiento.
   - También afecta a las capas de normalización, como Batch Normalization, para que utilicen estadísticas de mini-lotes (batches) durante el entrenamiento.

**model.eval()**:
   - Cuando se llama a `model.eval()`, el modelo se coloca en modo de evaluación o inferencia.
   - En este modo, las capas de dropout se desactivan, lo que significa que no se aplican durante la evaluación. Esto garantiza que el modelo produzca resultados deterministas y coherentes durante la inferencia.
   - Las capas de normalización, como Batch Normalization, utilizan estadísticas acumuladas durante el entrenamiento (en lugar de estadísticas de mini-lotes) para garantizar una evaluación coherente y precisa.


En la práctica, es común utilizar `model.train()` antes de cada época de entrenamiento y `model.eval()` antes de realizar inferencia o evaluación en un modelo entrenado. Cambiar entre estos modos es esencial para garantizar que el modelo se comporte correctamente en diferentes etapas del proceso de entrenamiento y evaluación.

In [11]:
import time

# Hyperparameters
EPOCHS = 10  # epoch
LR = 5  # learning rate

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

def train(dataloader):
    model.train()
    total_acc, total_count, max_acc = 0, 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()

        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| {:5d} batches '
                  '| accuracy {:8.3f}'.format(idx, total_acc / total_count))

            if max_acc < total_acc / total_count:
                max_acc = total_acc / total_count
                
            total_acc, total_count = 0, 0
            start_time = time.time()
    return max_acc


def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            predicted_label = model(text)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

In [12]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()

    accu_train = train(train_dataloader)
    accu_val = evaluate(test_dataloader)

    #if accu_train > accu_val:
    #    scheduler.step()
    
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

/home/adrian/miniconda3/envs/PLN/lib/python3.11/site-packages/torch/utils/data/datapipes/iter/combining.py:333: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


|   500 batches | accuracy    0.254
|  1000 batches | accuracy    0.261
|  1500 batches | accuracy    0.400
-----------------------------------------------------------
| end of epoch   1 | time: 91.24s | valid accuracy    0.517 
-----------------------------------------------------------
|   500 batches | accuracy    0.542
|  1000 batches | accuracy    0.596
|  1500 batches | accuracy    0.650
-----------------------------------------------------------
| end of epoch   2 | time: 81.67s | valid accuracy    0.695 
-----------------------------------------------------------
|   500 batches | accuracy    0.749
|  1000 batches | accuracy    0.794
|  1500 batches | accuracy    0.825
-----------------------------------------------------------
| end of epoch   3 | time: 81.96s | valid accuracy    0.825 
-----------------------------------------------------------
|   500 batches | accuracy    0.831
|  1000 batches | accuracy    0.850
|  1500 batches | accuracy    0.866
-------------------------

---

## **Práctica 1**

Modifica el código anterior para adaptar el modelo LSTM al uso de *embeddings* preentrenados. Para ello, usa <code>from torchtext.vocab import GloVe</code> y elige el conjunto de *embeddings* GloVe que prefieras. Puedes encontrar más información en https://pytorch.org/text/stable/vocab.html#torchtext.vocab.GloVe

Verifica si se produce una mejora en la precisión del modelo. ¿Qué ocurre si usas un conjunto de *embeddings* preentrenados de diferentes tamaños?



---

In [15]:
#Importamos el embeding preentrenado 
from torchtext.vocab import GloVe

glove = GloVe(name = "6B", dim = 300) #Descargamos el Embedding con tantas dimenciones como .

.vector_cache/glove.6B.zip: 0.00B [01:43, ?B/s]


FileNotFoundError: [Errno 2] No such file or directory: '.vector_cache/glove.6B.zip'